In [1]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.9/496.9 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 13.3 MB/s eta 0:00:00


In [2]:
## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
## connecting to the google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [5]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images
    

    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)

    
    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.png',f"{train_path_img}/" + filex + '.png' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')
        
    

    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")
    
    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".png", f"{train_path_img}/" + filex + '.png')
            
        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")
    
        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")
    


    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.png', f"{val_path_img}/" + filex + '.png' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")
    
    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/PureCode/dataset_d1/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 36 images---


  0%|          | 0/29 [00:00<?, ?it/s]

------ Training data created with 80% split 29 images -------


  0%|          | 0/7 [00:00<?, ?it/s]

------ Testing data created with a total of 7 images ----------
------ TASK COMPLETED -------


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.54 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 25.5/78.2 GB disk)


In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/PureCode/dataset.yaml epochs=100 imgsz=640 batch=8 project=/content/drive/MyDrive/PureCode/training_results name=psd1 

100% 21.5M/21.5M [00:00<00:00, 276MB/s]
Ultralytics YOLOv8.0.54 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/PureCode/dataset.yaml, epochs=100, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/PureCode/training_results, name=psd1, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=

Inference

In [ ]:
#from ultralytics import YOLO

# Load a model
#model = YOLO('/content/drive/MyDrive/PureCode/training_results/psd12/weights/best.pt')  # load a pretrained model (recommended for training)

In [ ]:
!yolo predict model='/content/drive/MyDrive/PureCode/training_results/psd1/weights/best.pt' conf=0.8 source='/content/drive/MyDrive/PureCode/infer_data/IMAGES' save_txt=True save_crop=True

Ultralytics YOLOv8.0.54 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11127519 parameters, 0 gradients, 28.4 GFLOPs

image 1/300 /content/drive/MyDrive/PureCode/infer_data/IMAGES/01357379-D6C2-4B01-AA66-C8790642D0B1.png: 320x640 2 btn-inactives, 1 btn-active, 3 btn-reds, 1 btn-orange, 2 btn-greens, 14.5ms
image 2/300 /content/drive/MyDrive/PureCode/infer_data/IMAGES/01B6145F-B88B-4FF8-B7D7-BB8A321EDA85.png: 320x640 1 btn-inactive, 1 btn-active, 1 btn-red, 1 btn-green, 11.4ms
image 3/300 /content/drive/MyDrive/PureCode/infer_data/IMAGES/02C4C780-79EA-43F1-B2AE-3A2C757F874C.png: 320x640 3 btn-inactives, 1 btn-active, 1 btn-red, 2 btn-greens, 11.3ms
image 4/300 /content/drive/MyDrive/PureCode/infer_data/IMAGES/03806667-47A8-4965-8E93-21E205D5E8A7.png: 320x640 2 btn-inactives, 1 btn-active, 5 btn-reds, 1 btn-orange, 63.4ms
image 5/300 /content/drive/MyDrive/PureCode/infer_data/IMAGES/042B145D-1E0D-47CB-B16C-7F49B64DFF53.png: 320x640 4 btn

Validation

In [7]:
!yolo val model=/content/drive/MyDrive/PureCode/training_results/psd1/weights/best.pt data=/content/drive/MyDrive/PureCode/dataset.yaml

Ultralytics YOLOv8.0.54 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11127519 parameters, 0 gradients, 28.4 GFLOPs
100% 755k/755k [00:00<00:00, 17.7MB/s]
val: Scanning /content/yolo_data/labels/val... 7 images, 0 backgrounds, 0 corrupt: 100% 7/7 [00:00<00:00, 501.96it/s]
val: New cache created: /content/yolo_data/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 1/1 [00:01<00:00,  1.87s/it]
                   all          7         68      0.989          1      0.995       0.83
          btn-inactive          7         20      0.992          1      0.995      0.884
            btn-active          7          7      0.978          1      0.995      0.861
               btn-red          7         10      0.988          1      0.995      0.799
            btn-orange          7         18      0.997          1      0.995      0.821
             btn-green          7         13  